## Loading the Data

In [1]:
from datasets import load_from_disk
dataset = load_from_disk('ds')

In [2]:
dataset['train'][0]

{'Tibetan': 'ཞི་དང་བཀྲ་ཤིས་བདེ་ལེགས་ཤིང་།།',
 'Phonetic': 'zhi dang tashi delek shing',
 'English': 'Bring peace good fortune happiness and wellbeing',
 'Tags': ['Sikkim',
  'Offering',
  'Tibetan Masters',
  'Jamyang Khyentse Chökyi Lodrö']}

## Load Unfinetuned Tokenizer, Model, and Data Collator

In [3]:
from transformers import AutoTokenizer, DataCollatorForSeq2Seq, AutoModelForSeq2SeqLM

checkpoint = "google-t5/t5-small"
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint, device_map="cuda:0")
tokenizer = AutoTokenizer.from_pretrained('my-tokenizer')
model.resize_token_embeddings(len(tokenizer))
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint)

## Preprocess Data

The dataset can now be tokenized for training.

In [4]:
def translation_preprocess_function(examples):

    # Prepare translation inputs and targets
    translation_inputs = ['Transcribe Tibetan: ' + example for example in examples['Tibetan']]
    translation_targets = [example for example in examples['Phonetic']]
    
    # Tokenize translation inputs and targets
    translation_model_inputs = tokenizer(translation_inputs, text_target=translation_targets, 
                                         max_length=256, truncation=False, padding="max_length")
    
    
    return translation_model_inputs


In [5]:
tokenized_dataset = dataset.map(translation_preprocess_function, batched=True)

Map:   0%|          | 0/96772 [00:00<?, ? examples/s]

Map:   0%|          | 0/10753 [00:00<?, ? examples/s]

## Train the Model

Finally, we can train the model. Note that the optimizer used is Adafactor. This is the optimizer that is preferred for translation tasks and for the T5 model in general. The transformers api includes a built in version of Adafactor, but I define it separately here so that we can optimize it with the 'accelerate' library.

In [6]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer, Adafactor, EarlyStoppingCallback
from accelerate import Accelerator

accelerator = Accelerator()

optimizer = Adafactor(
    model.parameters(), 
    scale_parameter=True, 
    relative_step=False, 
    warmup_init=False, 
    lr=3e-4
)

model, optimizer = accelerator.prepare(model, optimizer)

In [7]:
import numpy as np
import evaluate

# Load BLEU and CHRF metrics
bleu_metric = evaluate.load("sacrebleu")
chrf_metric = evaluate.load("chrf")
ter_metric = evaluate.load("ter")

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    
    # Decode predictions and labels
    preds = np.where(preds != -100, preds, tokenizer.pad_token_id)
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Postprocess text
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    # Compute BLEU score
    bleu_result = bleu_metric.compute(predictions=decoded_preds, references=decoded_labels)
    bleu_score = bleu_result["score"]

    # Compute CHRF score
    chrf_result = chrf_metric.compute(predictions=decoded_preds, references=decoded_labels)
    chrf_score = chrf_result["score"]

    # Compute TER score
    ter_result = ter_metric.compute(predictions=decoded_preds, references=decoded_labels)
    ter_score = ter_result["score"]

    # Return rounded results
    metrics = {
        "bleu": round(bleu_score, 4),
        "chrf": round(chrf_score, 4),
        "ter": round(ter_score, 4)
    }

    #print("Computed Metrics:", metrics)

    return metrics

In [8]:
%env WANDB_PROJECT=context-experiment

env: WANDB_PROJECT=context-experiment


In [9]:
training_args = Seq2SeqTrainingArguments(
    output_dir=f"no-context-transliteration",
    auto_find_batch_size=True,
    predict_with_generate=True,
    fp16=False,
    push_to_hub=False,
    eval_strategy='epoch',
    save_strategy='epoch',
    num_train_epochs=5
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['test'],
    tokenizer=tokenizer,
    optimizers=(optimizer, None),
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

trainer.train()

/tmp/ipykernel_15172/3087891381.py:12: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: billingsmoore. Use `wandb login --relogin` to force relogin


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss,Bleu,Chrf,Ter
1,0.177300,0.160819,5.163300,14.720000,98.363200
2,0.155000,0.134901,7.614400,19.735900,92.362600
3,0.134400,0.118220,9.584700,24.004600,87.833300
4,0.124500,0.107917,12.053100,28.123700,82.911100
5,0.119200,0.104547,12.875400,29.384200,81.280200


TrainOutput(global_step=60485, training_loss=0.15483158796180663, metrics={'train_runtime': 14080.1525, 'train_samples_per_second': 34.365, 'train_steps_per_second': 4.296, 'total_flos': 3.274324203012096e+16, 'train_loss': 0.15483158796180663, 'epoch': 5.0})